## Focus sur la fréquentation des stations voyageurs

Ce jeu de données (`Fréquentation en gares`) représente la fréquentation annuelle de l'ensemble des 3000 gares voyageurs de 2015 à 2021.

Ces informations sont issues d’une part, des données de billetterie pour le trafic national et régional hors Ile-de-France et d’autre part, d’une extrapolation pour le trafic régional Ile-de-France à partir des comptages effectués tous les 3 à 4 ans et publiés par ailleurs en Open Data.

A noter, pour les années 2020 et 2021 peu de comptages ont été réalisés pour le trafic régional Ile-de-France en raison de la crise sanitaire. La fréquentation a été calculée en relatif par rapport aux évolutions des validations en gare observées depuis 2019.


Une estimation de la proportion de non voyageurs est réalisée à partir des enquêtes en gare. En l'absence d’enquête, un taux par défaut est défini (20% pour les gares a et b hors Ile-de-France et 0% pour les gares b Ile-de-France et c).


La méthodologie de calcul se précise année après année avec une reprise de l'historique pouvant expliquer des variations dans les données publiées d'une année sur l'autre. Pour toute question, vous pouvez nous l'adresser au travers du formulaire de contact du portail Open Data.

***Informations du site open data de la [SNCF](https://ressources.data.sncf.com/explore/dataset/frequentation-gares/information/?disjunctive.nom_gare&disjunctive.code_postal&sort=nom_gare)***

### Récupération des données de fréquentation des gares

In [2]:
import geopandas as gpd
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt
import cartiflette.s3

In [1]:
url_freq = "https://ressources.data.sncf.com/api/v2/catalog/datasets/frequentation-gares/exports" + "/json?limit=-1" + "&offset=0" + "&lang=fr" + "&timezone=Europe%2FParis" # url de l'API pour acccéder aux données
dataFreq = rq.get(url_freq)
dataFreq = dataFreq.json() 
dataFreq = pd.json_normalize(dataFreq)

NameError: name 'rq' is not defined

In [ ]:
dataFreq.shape
dataFreq.head()
dataFreq.tail()

Comme on peut le voir, les totaux du nombre de voyageurs et du nombre de "**non-voyageurs**" sont fournis de 2015 à 2022, soit sur les sept(07) dernière années.

Dans le jeu de données , 
* **total_voyageurs** représsente le nombre total de voyageurs
* **total_voyageurs_non_voyageurs** représente le nombre total de voyageurs, 